In [1]:
import pandas as pd
import re
from pathlib import Path

In [5]:
#Configuration
dstFolder = "../data/raw/data.gouv/"
url = "https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_01_previous-1950-2023_RR-T-Vent.csv.gz"

In [3]:
dfUrl = pd.read_csv(dstFolder+"urls_stables.csv")
dfUrl = dfUrl[dfUrl['nom'].str.contains('vent', case=False)].copy()

dfUrl["debut"] = 0
dfUrl["fin"] = 0

for index, row in dfUrl.iterrows():
    r = re.search(r'(\d{4})-(\d{4})', row["nom"])
    dfUrl.loc[index, "debut"] = int(r.group(1))
    dfUrl.loc[index, "fin"] = int(r.group(2))

dfUrl.to_csv(dstFolder+"urls_stables_consolide.csv", index=False)

In [6]:
df = pd.read_csv(url, compression='gzip', sep=';')  
df2 = df[["TX","TN","LAT","LON","AAAAMMJJ","ALTI"]]
dff = df2[(df2["TX"] > 0) & (df2["TN"] > 0)]
dff

,TX,TN,LAT,LON,AAAAMMJJ,ALTI
10134,7.4,0.6,46.266667,5.650000,19691105,535
10137,11.7,5.0,46.266667,5.650000,19691108,535
10138,10.3,6.5,46.266667,5.650000,19691109,535
10139,15.4,5.0,46.266667,5.650000,19691110,535
10140,17.3,7.6,46.266667,5.650000,19691111,535
...,...,...,...,...,...,...
946653,6.3,0.2,45.906833,5.718167,20180318,603
946659,14.9,0.7,45.906833,5.718167,20180324,603
946660,14.9,0.7,45.906833,5.718167,20180325,603
946663,12.0,3.0,45.906833,5.718167,20180328,603


In [12]:
test = dff[(dff["AAAAMMJJ"] == 19691105)]
test

,TX,TN,LAT,LON,AAAAMMJJ,ALTI
10134,7.4,0.6,46.266667,5.650000,19691105,535
35534,11.0,3.5,45.833333,5.083333,19691105,197
157693,11.0,7.0,46.206667,5.221667,19691105,240
234139,9.5,4.3,45.976500,5.329333,19691105,250
473242,7.8,4.9,46.225000,5.541667,19691105,430
552579,9.0,2.0,46.038500,5.044333,19691105,280
635742,9.0,1.5,45.981667,4.801667,19691105,270


Pour récupérer la liste des urls du site : https://www.data.gouv.fr/datasets/donnees-climatologiques-de-base-quotidiennes/

curl "https://www.data.gouv.fr/api/1/datasets/donnees-climatologiques-de-base-quotidiennes/" | jq -r '.resources[] | [.title, .url] | @csv' > urls_stables.csv

In [ ]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20949 entries, 26221 to 110629
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TX        20949 non-null  float64
 1   TN        20949 non-null  float64
 2   LAT       20949 non-null  float64
 3   LON       20949 non-null  float64
 4   AAAAMMJJ  20949 non-null  int64  
 5   ALTI      20949 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 1.1 MB


In [ ]:
#Téléchargement
dfSources = pd.read_csv(dstFolder+"urls_stables_consolide.csv")
#On garde que ceux > 1950
dfSources = dfSources[(dfSources["fin"] > 1950)].copy()
dfSources


,nom,url,debut,fin
1,QUOT_departement_01_periode_1950-2023_RR-T-Vent,https://object.files.data.gouv.fr/meteofrance/...,1950,2023
2,QUOT_departement_01_periode_2024-2025_RR-T-Vent,https://object.files.data.gouv.fr/meteofrance/...,2024,2025
4,QUOT_departement_02_periode_1950-2023_RR-T-Vent,https://object.files.data.gouv.fr/meteofrance/...,1950,2023
5,QUOT_departement_02_periode_2024-2025_RR-T-Vent,https://object.files.data.gouv.fr/meteofrance/...,2024,2025
7,QUOT_departement_03_periode_1950-2023_RR-T-Vent,https://object.files.data.gouv.fr/meteofrance/...,1950,2023
...,...,...,...,...
308,QUOT_departement_987_periode_1950-2023_RR-T-Vent,https://object.files.data.gouv.fr/meteofrance/...,1950,2023
309,QUOT_departement_987_periode_2024-2025_RR-T-Vent,https://object.files.data.gouv.fr/meteofrance/...,2024,2025
311,QUOT_departement_988_periode_1950-2023_RR-T-Vent,https://object.files.data.gouv.fr/meteofrance/...,1950,2023
312,QUOT_departement_988_periode_2024-2025_RR-T-Vent,https://object.files.data.gouv.fr/meteofrance/...,2024,2025


In [ ]:
fileIndex = 1
for index, row in dfSources.iterrows():
    print(f"Traitement du fichier {fileIndex} {row["nom"]}")
    fichier = Path(f"{dstFolder}{fileIndex}.csv")
    if fichier.exists():
        print("Fichier déjà existant, on continue")
        fileIndex = fileIndex +1
        continue
    
    df = pd.read_csv(row["url"], compression='gzip', sep=';')  
    df2 = df[["TX","TN","LAT","LON","AAAAMMJJ","ALTI"]]
    dff = df2[(df2["TX"] > 0) & (df2["TN"] > 0)]
    dff.to_csv(f"{dstFolder}{fileIndex}.csv")
    print("Saved !")
    fileIndex = fileIndex +1
    

Traitement du fichier 1 QUOT_departement_01_periode_1950-2023_RR-T-Vent
Fichier déjà existant, on continue
Traitement du fichier 2 QUOT_departement_01_periode_2024-2025_RR-T-Vent
Fichier déjà existant, on continue
Traitement du fichier 3 QUOT_departement_02_periode_1950-2023_RR-T-Vent
Fichier déjà existant, on continue
Traitement du fichier 4 QUOT_departement_02_periode_2024-2025_RR-T-Vent
Fichier déjà existant, on continue
Traitement du fichier 5 QUOT_departement_03_periode_1950-2023_RR-T-Vent
Fichier déjà existant, on continue
Traitement du fichier 6 QUOT_departement_03_periode_2024-2025_RR-T-Vent
Fichier déjà existant, on continue
Traitement du fichier 7 QUOT_departement_04_periode_1950-2023_RR-T-Vent
Fichier déjà existant, on continue
Traitement du fichier 8 QUOT_departement_04_periode_2024-2025_RR-T-Vent
Fichier déjà existant, on continue
Traitement du fichier 9 QUOT_departement_05_periode_1950-2023_RR-T-Vent
Fichier déjà existant, on continue
Traitement du fichier 10 QUOT_departe

In [ ]:
import pyarrow as pa

# Avant votre code, ajoutez :
try:
    pa.unregister_extension_type("pandas.period")
except KeyError:
    pass

In [ ]:
#Consolidation

dataFrames = []

for i in range(211):
    i = i+1
    print(f"Traitement fichier {i}")
    df = pd.read_csv(f"{dstFolder}{i}.csv")
    dataFrames.append(df)

dfAll = pd.concat(dataFrames, ignore_index=True)
dfAll.to_parquet(f"{dstFolder}meteo.parquet", engine='fastparquet')

Traitement fichier 1
Traitement fichier 2
Traitement fichier 3
Traitement fichier 4
Traitement fichier 5
Traitement fichier 6
Traitement fichier 7
Traitement fichier 8
Traitement fichier 9
Traitement fichier 10
Traitement fichier 11
Traitement fichier 12
Traitement fichier 13
Traitement fichier 14
Traitement fichier 15
Traitement fichier 16
Traitement fichier 17
Traitement fichier 18
Traitement fichier 19
Traitement fichier 20
Traitement fichier 21
Traitement fichier 22
Traitement fichier 23
Traitement fichier 24
Traitement fichier 25
Traitement fichier 26
Traitement fichier 27
Traitement fichier 28
Traitement fichier 29
Traitement fichier 30
Traitement fichier 31
Traitement fichier 32
Traitement fichier 33
Traitement fichier 34
Traitement fichier 35
Traitement fichier 36
Traitement fichier 37
Traitement fichier 38
Traitement fichier 39
Traitement fichier 40
Traitement fichier 41
Traitement fichier 42
Traitement fichier 43
Traitement fichier 44
Traitement fichier 45
Traitement fichier 